In [ ]:
import tensorflow as tf
from sklearn.datasets import make_classification
import numpy as np

# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_classes=2, random_state=42)

# Split dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert numpy arrays to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define a simple feedforward neural network
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(20,)),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')
    ])
    return model

# Create an instance of the model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#1. Temporal Smoothing with Accuracy Tracking

Temporal smoothing applies a penalty to the model's predictions based on the smoothness of the prediction trajectory over time. It helps in reducing model sensitivity to noisy inputs.

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Define a custom loss function with temporal smoothing regularization
def temporal_smoothing_loss(y_true, y_pred):
    smoothness_penalty = tf.reduce_mean(tf.square(y_pred[:, 1:] - y_pred[:, :-1]))
    return tf.losses.sparse_categorical_crossentropy(y_true, y_pred) + smoothness_penalty

# Compile your model using the custom loss function and accuracy metric
model.compile(optimizer='adam', loss=temporal_smoothing_loss, metrics=['accuracy'])

# Define a function to calculate accuracy on a dataset
def calculate_accuracy(model, dataset):
    y_true = []
    y_pred = []
    for x_batch, y_batch in dataset:
        predictions = model.predict(x_batch)
        y_true.extend(y_batch.numpy())
        y_pred.extend(tf.argmax(predictions, axis=1).numpy())
    return accuracy_score(y_true, y_pred)

# Train the model with accuracy tracking
num_epochs = 10
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dataset:
        model.train_on_batch(x_batch, y_batch)

    # Calculate training accuracy
    train_accuracy = calculate_accuracy(model, train_dataset)
    print(f'Epoch {epoch + 1}, Training Accuracy: {train_accuracy}')

1/1 [==============================] - 0s 59ms/step
Epoch 1, Training Accuracy: 0.9125
1/1 [==============================] - 0s 19ms/step
Epoch 2, Training Accuracy: 0.92
1/1 [==============================] - 0s 31ms/step
Epoch 3, Training Accuracy: 0.93
1/1 [==============================] - 0s 75ms/step
Epoch 4, Training Accuracy: 0.935
1/1 [==============================] - 0s 20ms/step
Epoch 5, Training Accuracy: 0.945
1/1 [==============================] - 0s 30ms/step
Epoch 6, Training Accuracy: 0.94
1/1 [==============================] - 0s 23ms/step
Epoch 7, Training Accuracy: 0.93875
1/1 [==============================] - 0s 20ms/step
Epoch 8, Training Accuracy: 0.95125
1/1 [==============================] - 0s 22ms/step
Epoch 9, Training Accuracy: 0.955
1/1 [==============================] - 0s 29ms/step
Epoch 10, Training Accuracy: 0.95


#2. DropConnect

DropConnect is an extension of dropout where instead of dropping units (neurons), a randomly selected subset of weights within a layer are set to zero during training.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define a simple feedforward neural network
def create_model2():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(20,)),
        Dropout(0.5),
        Dense(32, activation='relu'),
    ])
    return model

# Create an instance of the model
model = create_model2()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define a custom layer with DropConnect
class DropConnect(tf.keras.layers.Layer):
    def __init__(self, rate=0.5):
        super(DropConnect, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            mask = tf.random.uniform(tf.shape(inputs)) > self.rate
            masked_inputs = inputs * tf.cast(mask, tf.float32)
            return masked_inputs
        return inputs

# Use DropConnect in the model
model.add(DropConnect(rate=0.5))
model.add( Dense(2, activation='softmax'))

# Train the model
model.fit(train_dataset, epochs=50)

Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: 0.7772 - accuracy: 0.5537
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6682 - accuracy: 0.6162
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5867 - accuracy: 0.6800
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5199 - accuracy: 0.7400
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5173 - accuracy: 0.7312
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4731 - accuracy: 0.7875
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4473 - accuracy: 0.7688
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4040 - accuracy: 0.8163
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3745 - accuracy: 0.8400
Epoch 10/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3675 - accuracy: 0.8438
Epoch 11/50
25/25 [

#3. Stochastic Depth

Stochastic Depth randomly skips entire layers during training, effectively training shorter networks within the full architecture. This regularizes the model and encourages robustness.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

# Define a custom layer with stochastic layer skipping
class StochasticDepth(Layer):
    def __init__(self, skip_prob=0.5):
        super(StochasticDepth, self).__init__()
        self.skip_prob = skip_prob

    def call(self, inputs, training=None):
        if training:
            if tf.random.uniform(shape=[]) < self.skip_prob:
                return inputs  # Skip layer
            else:
                return inputs  # Return inputs if not skipped during training
        else:
            return inputs  # Return inputs if not in training mode

# Create an instance of the model
model = create_model2()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Use StochasticDepth in the model
model.add(StochasticDepth(skip_prob=0.5))
model.add(Dense(2, activation='softmax'))

# Train the model
model.fit(train_dataset, epochs=50)

Epoch 1/50
25/25 [==============================] - 2s 6ms/step - loss: 0.8956 - accuracy: 0.5750
Epoch 2/50
25/25 [==============================] - 0s 6ms/step - loss: 0.5952 - accuracy: 0.6913
Epoch 3/50
25/25 [==============================] - 0s 6ms/step - loss: 0.4756 - accuracy: 0.7738
Epoch 4/50
25/25 [==============================] - 0s 6ms/step - loss: 0.4365 - accuracy: 0.7962
Epoch 5/50
25/25 [==============================] - 0s 5ms/step - loss: 0.3900 - accuracy: 0.8338
Epoch 6/50
25/25 [==============================] - 0s 5ms/step - loss: 0.3535 - accuracy: 0.8462
Epoch 7/50
25/25 [==============================] - 0s 5ms/step - loss: 0.3348 - accuracy: 0.8475
Epoch 8/50
25/25 [==============================] - 0s 4ms/step - loss: 0.2842 - accuracy: 0.8725
Epoch 9/50
25/25 [==============================] - 0s 4ms/step - loss: 0.3004 - accuracy: 0.8637
Epoch 10/50
25/25 [==============================] - 0s 5ms/step - loss: 0.3022 - accuracy: 0.8650
Epoch 11/50
25/25 [

#4. Orthogonal Regularization

Orthogonal regularization enforces orthogonal constraints on weight matrices, encouraging feature diversity and reducing redundancy.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import numpy as np

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Custom layer with orthogonal weight regularization
class OrthogonalRegularizationLayer(layers.Layer):
    def __init__(self, units):
        super(OrthogonalRegularizationLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                  initializer='random_normal',
                                  trainable=True,
                                  regularizer=self.orthogonal_regularizer)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)

    def orthogonal_regularizer(self, w):
        w = tf.transpose(w)
        identity = tf.eye(tf.shape(w)[0])
        ortho_loss = tf.reduce_mean(tf.square(tf.matmul(w, w, transpose_b=True) - identity))
        return ortho_loss

# Build a simple model using the custom layer
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    OrthogonalRegularizationLayer(128),
    layers.ReLU(),
    layers.Dense(10, activation='softmax')
])

# Compile and train the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 17s 8ms/step - loss: 0.2827 - accuracy: 0.9266 - val_loss: 0.1733 - val_accuracy: 0.9569
Epoch 2/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.1558 - accuracy: 0.9628 - val_loss: 0.1331 - val_accuracy: 0.9659
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1245 - accuracy: 0.9719 - val_loss: 0.1243 - val_accuracy: 0.9707
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1077 - accuracy: 0.9753 - val_loss: 0.1073 - val_accuracy: 0.9741
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0961 - accuracy: 0.9776 - val_loss: 0.1106 - val_accuracy: 0.9719


#5. Adversarial Training (FGSM)

Adversarial training involves augmenting the training data with adversarial examples generated to fool the model, which helps in improving the model's robustness against adversarial attacks.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, datasets

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reduce training set size for faster demonstration
train_images = train_images[:2000]
train_labels = train_labels[:2000]

# Define the model
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Function to generate adversarial examples using FGSM
def generate_adversarial_example(model, image, label, epsilon=0.2):
    image = tf.convert_to_tensor(image)  # Convert to tensor if not already
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    label = tf.convert_to_tensor([label])  # Convert label to tensor

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = losses.sparse_categorical_crossentropy(label, prediction)

    gradient = tape.gradient(loss, image)
    perturbation = tf.sign(gradient)
    adversarial_image = image + epsilon * perturbation
    adversarial_image = tf.clip_by_value(adversarial_image, 0, 1)  # Clip to valid image range

    return adversarial_image[0]  # Remove batch dimension and return

# Adversarial training using model.fit
epochs = 5
epsilon = 0.2  # FGSM perturbation magnitude

# Prepare adversarial examples for training
adversarial_images = []
for image, label in zip(train_images, train_labels):
    adv_image = generate_adversarial_example(model, image, label, epsilon)
    adversarial_images.append(adv_image)

adversarial_images = tf.convert_to_tensor(adversarial_images)

# Train the model using adversarial examples
model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels))

# Final evaluation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Final test accuracy: {test_acc}")

Epoch 1/5
63/63 [==============================] - 2s 16ms/step - loss: 1.1240 - accuracy: 0.6880 - val_loss: 0.5804 - val_accuracy: 0.8441
Epoch 2/5
63/63 [==============================] - 1s 14ms/step - loss: 0.4478 - accuracy: 0.8855 - val_loss: 0.4302 - val_accuracy: 0.8778
Epoch 3/5
63/63 [==============================] - 1s 12ms/step - loss: 0.3162 - accuracy: 0.9155 - val_loss: 0.3829 - val_accuracy: 0.8906
Epoch 4/5
63/63 [==============================] - 2s 25ms/step - loss: 0.2577 - accuracy: 0.9360 - val_loss: 0.3723 - val_accuracy: 0.8912
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3420 - accuracy: 0.8989
Final test accuracy: 0.8988999724388123


#6. Dynamic Weight Averaging

Dynamic Weight Averaging maintains a running average of model parameters during training, allowing the model to generalize better by averaging over multiple checkpoints.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, datasets
import numpy as np

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the model
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Custom callback for dynamic weight averaging
class DynamicWeightAveragingCallback(tf.keras.callbacks.Callback):
    def __init__(self, decay=0.99):
        super(DynamicWeightAveragingCallback, self).__init__()
        self.decay = decay

    def on_train_begin(self, logs=None):
        # Initialize averaged weights to the current model weights
        self.averaged_weights = [tf.Variable(w) for w in self.model.get_weights()]

    def on_batch_end(self, batch, logs=None):
        # Update averaged weights after each batch
        current_weights = self.model.get_weights()
        for avg_weight, weight in zip(self.averaged_weights, current_weights):
            avg_weight.assign(self.decay * avg_weight + (1 - self.decay) * weight)

# Train the model with dynamic weight averaging
epochs = 5

# Create an instance of the custom callback
dynamic_weight_callback = DynamicWeightAveragingCallback()

# Fit the model with the callback
model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels),
          callbacks=[dynamic_weight_callback])

# Use averaged weights for inference
model.set_weights([avg.numpy() for avg in dynamic_weight_callback.averaged_weights])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy (with dynamic weight averaging): {test_acc}")

Epoch 1/5
   4/1875 [..............................] - ETA: 44s - loss: 2.2995 - accuracy: 0.1719  

1875/1875 [==============================] - 19s 9ms/step - loss: 0.2560 - accuracy: 0.9261 - val_loss: 0.1365 - val_accuracy: 0.9613
Epoch 2/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1106 - accuracy: 0.9672 - val_loss: 0.0931 - val_accuracy: 0.9704
Epoch 3/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0777 - accuracy: 0.9766 - val_loss: 0.0842 - val_accuracy: 0.9754
Epoch 4/5
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0579 - accuracy: 0.9819 - val_loss: 0.0808 - val_accuracy: 0.9754
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0692 - accuracy: 0.9799
Test accuracy (with dynamic weight averaging): 0.9799000024795532
